# Getting Started

## API authentication

API calls are authenticated by an account ID (which is not really secret), an API key (which definitely should be kept secret) and the key's role (Viewer or Editor). Storing the key directly within an API script is a very bad idea, so a common way to make the key available to a script without encoding it inside the script is to create an environment variable in which to hold the key, and then have the script read the key from the environment variable at run time.

Henceforth all the notebooks in this repo will use the following code to load an API key from an environment variable called "CATO_API_KEY". If you don't know how to set up and manage environment variables on your platform of choice, now is a good time to learn. If for whatever reason you're unwilling or unable to do this right now, you should uncomment the "THIS IS A BAD IDEA" line, and paste your key directly into the notebook.

In [1]:
import os
CATO_API_KEY = os.environ.get("CATO_API_KEY", None)

#
# THIS IS A BAD IDEA
#
#CATO_API_KEY = "PASTE_KEY_HERE"

if CATO_API_KEY is not None:
    print("Successfully loaded API key.")
else:
    print("PROBLEM: couldn't load a key from the environment.")

Successfully loaded API key.


The account ID is visible in the Cato Management Application in the address bar, and on the **Administration / General Info** page, and is safe to hard-code into your scripts, although it can also be stored as an environment variable. From now on, all notebooks will include it as a hard-coded parameter which you replace with your own ID:

In [2]:
#
# Replace the 10434 with your own account ID
#
CATO_ACCOUNT_ID = 10434

## Making Cato API calls

The Cato API is GraphQL with a single URL for all customers, https://api.catonetworks.com/api/v1/graphql2. Each call consists of three elements:

* The operation name, which is a single string.
* A set of variables, which in Python is usually constructed as a dictionary. For simple calls the dictionary might only have one element; for more complex calls it can be nested to multiple levels.
* A query string in GraphQL's "Looks a bit like JSON but isn't really" format.

It is possible to interpolate the variable values directly into the query string, in which case no separate variables component is required, and for developers who are new to GraphQL, this can look like a simple and attractive option. It is, however, worth learning how to use parameterised queries with separate variables as data analytics usually requires multiple queries, and using parameters makes the job much easier. The three elements are supplied in the POST body, like this:
```
{"operationName":"operation name string", "variables":{the variable dictionary}, "query":"query string"}
```
The request needs two mandatory headers and one optional header:
* The API key is supplied as the value for the x-api-key header.
* The Content-Type must be "application/json".
* The optional header is to enable compression. 

Compression is particularly important when doing data analytics with Cato because Cato data, especially events, tend to be extremely compressible (up to 95%) so enabling compression results in much faster API calls and much less data transferred over the network. 

Many Cato customers use the **requests** module to make HTTP calls with Python. For these examples we will instead use the urllib module which ships with Python, to avoid having to install another third party import. There is no reason not to use requests in production, but occasionally we work with customers who are trying to run Python in locked-down and/or headless environments where installing third party modules can be problematic, so it can be useful to know that there is an alternative to fall back on.

Putting all of this together, one of the simplest API calls which can be made is accountSnapshot with only the account ID field in the response. This is a useful "ping" to check that everything is working, and that we can make API calls with a recognisable output. For this example we will not ask for compression, but for later notebooks where we are using a separate module for making API calls, compression will always be enabled.


In [3]:
import json
import ssl
import urllib.parse
import urllib.request

headers = {
    "x-api-key": CATO_API_KEY,
    "Content-Type": "application/json"
}

body = {
    "operationName": "accountSnapshot",
    "variables": {"accountID":CATO_ACCOUNT_ID},
    "query": '''
query accountSnapshot($accountID:ID!) {
    accountSnapshot(accountID:$accountID) {
        id
    }
}'''
}

request = urllib.request.Request(url='https://api.catonetworks.com/api/v1/graphql2', 
                                 data=json.dumps(body).encode("ascii"),
                                 headers=headers)
response = urllib.request.urlopen(request, context=ssl._create_unverified_context())

print(json.loads(response.read().decode("utf-8","replace")))

{'data': {'accountSnapshot': {'id': '10434'}}}


If that worked, you should see a line similar to:
```
{'data': {'accountSnapshot': {'id': '10434'}}}
```

Note the use of the ssl unverified context, which disables TLS inspection, in order to provide the best possible chance of the test query succeeding in this demo. In a production environment you might want to modify this line to make requests with TLS inspection enabled, and configure the infrastructure to support this (i.e. import any inspection certificates being used by legitimate inspection engines, or configure TLS bypass for Cato API requests).

Subsequent notebooks will import a locally-defined module which encapsulates the functionality needed to make API calls, together with more advanced features such as compression and error detection, in a single, easy-to-use object:

In [4]:
from cato import API

# Reuse the ID and key from the previous example
C = API(CATO_API_KEY)
variables = {"accountID":CATO_ACCOUNT_ID}
query = '''
query accountSnapshot($accountID:ID!) {
    accountSnapshot(accountID:$accountID) {
        id
    }
}'''
success, result = C.send("accountSnapshot", variables, query)
print(success, result)

True {'data': {'accountSnapshot': {'id': '10434'}}}


> Certificate validation with Python and the standard library can be tricky. Python doesn't always load the system's set of trusted CA certificates. The first example above gets around this by disabling certificate validation, which is not a good idea. The cato.API module uses the Python certifi module to load a curated set of CA certificates. If neither of these approaches work for you, the third party **requests** module is often better able to load the system CA certificates and may be a more suitable alternative for making Cato API requests using Python.